<a href="https://colab.research.google.com/github/vitroid/PythonTutorials/blob/master/9%20Preparation/Redlich_Kwong_CP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# van der Waals状態方程式の臨界点

In [ ]:
from sympy import *
from sympy.abc import *
vdW = (p+a/V**2)*(V-b)-k*T
solve([vdW, vdW.diff(V), vdW.diff(V,2)], [p, V, T])

[(a/(27*b**2), 3*b, 8*a/(27*b*k))]

## Redlich-Kwong状態方程式の臨界点


状態方程式の変曲点は、熱力学では臨界点と呼びます。体積変化に対して圧力が変動しないということは、圧縮率が無限大になり、体積が大きくゆらぐことを示唆しています。圧縮率が無限大ということはまた、音速が0になることを意味します。

The inflection point of the state equation is called the critical point in thermodynamics.
As we see above, $\partial p/\partial V$ becomes zero, i.e.,  the compressibility $\partial V/\partial p$ becomes infinite.
At the critical point, the volume fluctuates greatly, and the sound velocity becomes zero.


In [1]:
import sympy
from sympy import *
from sympy.abc import *

# RK状態方程式
p=k*T/(V-b)-a/(sqrt(T)*V*(V+b))
# 微分と二階微分が同時に0になるのが臨界点。
p1=p.diff(V)
p2=p.diff(V,2)


In [2]:
p1

-T*k/(V - b)**2 + a/(sqrt(T)*V*(V + b)**2) + a/(sqrt(T)*V**2*(V + b))

In [3]:
p2

2*(T*k/(V - b)**3 - a/(sqrt(T)*V*(V + b)**3) - a/(sqrt(T)*V**2*(V + b)**2) - a/(sqrt(T)*V**3*(V + b)))

vdWの場合と同じ方法では解けなかったので、代わりに(a,b)について解く。

In [7]:
k,T,V,a,b = symbols("k T V a b", real=True, positive=True)
p = Symbol("p", real=True)
EOS=k*T/(V-b)-a/(sqrt(T)*V*(V+b))-p
solve((EOS.diff(V), EOS.diff(V,2)), (a,b))

[(2**(2/3)*T**(3/2)*V*k/(6 - 3*2**(2/3)), V*(-1 + 2**(1/3)))]

In [8]:
a0, b0 = _[0]
b0

V*(-1 + 2**(1/3))

bから、臨界体積$V_c$が得られる。

In [9]:
solve(b0-b, V)

[b/(-1 + 2**(1/3))]

In [10]:
Vc = _[0]
Vc

b/(-1 + 2**(1/3))

同じ要領で、a0の式からTcを求める。

In [17]:
solve(a-a0, T)

[(-3*a + 3*2**(1/3)*a)**(2/3)/(V**(2/3)*k**(2/3))]

In [18]:
Tc = _[0].subs(V, Vc)
Tc

(-1 + 2**(1/3))**(2/3)*(-3*a + 3*2**(1/3)*a)**(2/3)/(b**(2/3)*k**(2/3))

さいごに、TcとVcからpcを得る。

In [20]:
solve(EOS, p)

[(T**(3/2)*V**2*k + T**(3/2)*V*b*k - V*a + a*b)/(sqrt(T)*V*(V**2 - b**2))]

In [21]:
pc = _[0].subs(T, Tc).subs(V, Vc)
pc

k**(1/3)*(-1 + 2**(1/3))**(2/3)*(-a*b/(-1 + 2**(1/3)) + a*b + b*(-3*a + 3*2**(1/3)*a) + b*(-3*a + 3*2**(1/3)*a)/(-1 + 2**(1/3)))/(b**(2/3)*(-3*a + 3*2**(1/3)*a)**(1/3)*(-b**2 + b**2/(-1 + 2**(1/3))**2))

平方根、立方根を実数に変換し、Wikipediaと比較できるようにする。

In [23]:
pc.evalf()

0.0298943859926416*a**0.666666666666667*b**(-1.66666666666667)*k**0.333333333333333

In [24]:
Tc.evalf()

0.345039995853347*a**0.666666666666667*b**(-0.666666666666667)*k**(-0.666666666666667)

In [26]:
Vc.evalf()

3.84732210186307*b

[Wikipedia:Redlich-Kwong](https://en.wikipedia.org/wiki/Redlich%E2%80%93Kwong_equation_of_state)に一致したように思える。